# 방패

In [1]:
#-*-coding:utf-8-*-

# 연습문제 8-3: 유튜브 댓글 수집하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import openpyxl

from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

chrome_options = Options()
chrome_options.add_argument("--headless")


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("연습문제 : 유투브 영상의 댓글 수집하기")
print("=" *80)


query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")



# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )


#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
#path = "C:\\Temp\\chromedriver_240\\chromedriver.exe"
driver = webdriver.Chrome("C:\\Temp\\chromedriver_240\\chromedriver.exe",options=chrome_options)

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

#검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if reple_cnt < 20 :
        page_cnt = 1 
else :
        page_cnt = math.ceil(cnt/20)
        
if cnt > 20 :
           
        i = 1
        while ( i <=  page_cnt) :
            print("화면을 %s 회 아래로 스크롤 합니다" %i)
            scroll_down(driver)
            time.sleep(1)
            i += 1
      

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
          
          item.append(i['href'])
          count += 1

          if count == cnt :
              break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
          url_cnt += 1
          url = 'https://www.youtube.com/' +item[x]
          full_url.append(url)
          if url_cnt == cnt :
            break


play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]

wb=openpyxl.Workbook( )
wb.save(fx_name)


for y in range(0,len(full_url)) :
      driver.get(full_url[y])
      time.sleep(10)
      print("\n")
      print("\n")
      print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

      i = 1
      while ( i <=  page_cnt) :
      #print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(10)
        i += 1
      
      html = driver.page_source
      soup2 = BeautifulSoup(html, 'html.parser')
        
      t_count_0 = soup2.select('#count')
      t_count_1 = t_count_0[1].get_text()
      t_count_2 = t_count_1.replace(",","")
      t_count_3 = re.search("\d+",t_count_2)
      t_count_4 = int(t_count_3.group())

      t_title_1 = soup2.select('#info-contents')
      t_title_2 = t_title_1[0].find('h1').get_text()
      t_title_3 = t_title_2.translate(bmp_map).replace("\n","")
        
        
      try:
        t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()  #.get_text( ).replace("\n","")
      
      except AttributeError:
        continue
      
      else:
        t_view_2 = t_view_1.replace("," ,"")
        t_view_3 = re.search("\d+",t_view_2)
        print(t_view_1)
        t_view_4 = int(t_view_3.group())
        

#       t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()#.get_text( ).replace("\n","")
#       t_view_2 = t_view_1.replace("," ,"")
#       t_view_3 = re.search("\d+",t_view_2)
#       print(t_view_1)
#       t_view_4 = int(t_view_3.group())

      print("=" *80)
      print("%s 번째 동영상의 조회수는 %s 회 이고 수집할 댓글은 총 %s개 입니다" %(play_cnt,t_count_4,t_view_4))  
      print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
      print("=" *80)
      print("\n")
      print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
      print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
      print("\n")

 #Step 5. 화면을 스크롤해서 아래로 이동합니다

      #page_cnt = math.ceil(t_view_4 / 19)

      i = 1
      while ( i <=  page_cnt+1) :
        print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
        scroll_down(driver)
        time.sleep(0.5)
        i += 1

      #Step 6. 내용을 수집합니다

      time.sleep(2)

      html = driver.page_source
      soup3 = BeautifulSoup(html, 'html.parser')
        
      count = 0
      d2 = 0

      reple_result = soup3.select('#comments > #sections > #contents')

      for a in range(0,reple_cnt+1) :
              count += 1
              
              f = open(ff_name, 'a',encoding='UTF-8')
              f.write("\n")
              f.write("------------------------------------------------------------------"+"\n")

             # 댓글 작성자
              try : 
                writer = reple_result[0].select("#header-author > #author-text")[a].get_text().replace("\n","").strip()
              except IndexError :
                break
              else :
                print("\n")
                print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
                print("-" *70)

                # 유튜브 URL 주소
                print("1.URL 주소:",full_url[y])
                f.write("1.유투브 URL주소: " + full_url[y] + "\n")
                url2.append(full_url[y])
                
                print("2.댓글 작성자명:", writer)
                f.write("2.댓글 작성자명: " + writer + "\n")
                writer2.append(writer)
              
                # 댓글 작성일자
                wdate = reple_result[0].select("yt-formatted-string.published-time-text > a.yt-simple-endpoint")[a].get_text().replace("\n","")
            
                print("3.댓글 작성일자:",wdate)
                f.write("3.댓글 작성일자: " + wdate + "\n")
                wdate2.append(wdate)
              
                # 댓글 내용
                reple1 = reple_result[0].select('#expander > #content > #content-text')[a].get_text().replace("\n","")
            
                reple2=reple1.translate(bmp_map).replace("\n","")
                print("4.댓글 내용:",reple2)
                f.write("4.댓글 내용: " + reple2 + "\n")
                reple3.append(reple2)
                
              
              f.close( )
              
              if count == reple_cnt :
                  break
              
      time.sleep(2)       
      play_cnt += 1                

 #step 9. 엑셀 형태로 저장하기
    
              
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )


print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

time.sleep(3)

driver.close( )

 

연습문제 : 유투브 영상의 댓글 수집하기
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 산업재해보상보험
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:200
3.각 동영상에서 추출할 댓글은 몇건입니까?: 159
4.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\temp\):C:\\temp\\산업재해보상보험\\


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^
화면을 1 회 아래로 스크롤 합니다
화면을 2 회 아래로 스크롤 합니다
화면을 3 회 아래로 스크롤 합니다
화면을 4 회 아래로 스크롤 합니다
화면을 5 회 아래로 스크롤 합니다
화면을 6 회 아래로 스크롤 합니다
화면을 7 회 아래로 스크롤 합니다
화면을 8 회 아래로 스크롤 합니다
화면을 9 회 아래로 스크롤 합니다
화면을 10 회 아래로 스크롤 합니다


검색된 영상 중 200 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.
댓글 194개
1 번째 동영상의 조회수는 39776 회 이고 수집할 댓글은 총 194개 입니다
1 번째 동영상의 제목은 산재보험, 제대로 알고 보상 신청하자 입니다


1 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


1 번째 영상의 1 번째  댓글 
-----------------------------------------------------

4.댓글 내용: 감사합니다,덕분에 도움 됬습니다 월요일에 현장에서 무거운거 들다가 허리디스크 파열되서 입원하게 되었는데 허리 쪽이라 산재 처리를 신청할려고 하니 회사에서는 병원가서 밖에서 다쳤다고 얘기를 하라고 하네여 참...


1 번째 영상의 25 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: 공부합시다
3.댓글 작성일자: 4개월 전
4.댓글 내용: 고용인은 없고 일용직을 가끔씁니다. 산재 고용보험을 어디다 어떻게 가입합니까?  세무사사무실에서 업무를 같이 봐주는지요?


1 번째 영상의 26 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: 이동직
3.댓글 작성일자: 4개월 전
4.댓글 내용: 버스기사인데 운전으로인해서 왼쪽어깨가많이 아파서 산재처리받으려고하는데 가능할까요?


1 번째 영상의 27 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: 배혜빈
3.댓글 작성일자: 1년 전
4.댓글 내용: 안녕하세요 제가 직업이 운동선수로 되있어요 훈련도중에 다쳐서 수술을 하게됐는데 이런 부분은 어떻게 처리를 해야되나요 운동하다가 다쳤다고 해도 보상 받을수있나요?ㅠㅠ


1 번째 영상의 28 번째  댓글 
----------------------------------------------------------------------
1.URL 

3.댓글 작성일자: 1개월 전
4.댓글 내용: 근무중 낙하사고를 당한지 3년이지낫습니다2016년3월즘 다쳣눈데요산재처리를 못하고 두달치 월급받고 끝낫는데요다 나으면 재 출근하기로 하고두달치 월급받앗엇눈데요산재처리는 힘들더라도현재 장애가 남아있는데요후유장애신청도 불가 할까여?


1 번째 영상의 52 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: 산산
3.댓글 작성일자: 1개월 전
4.댓글 내용: 회사이전준비하다가 짐든상태로 더워서 땀닦앗더니 바로 눈아래 수포 생기고 또 눈다라끼생겨서 3년가까이 안과다니는데산재신청햇더니 경위내용을 네장정도는 제출요청하네요조금전 전화왓고 기타자세한 자료요청은 메일보냇다 하는데요치료비는 3십만원정도 들엇는데요요청자료번거로와 또 직장너무바빠 포기하고 싶은데 어쩌지요ㅜㅜ눈다라키와 눈부위염증으로안과에 30번정도나 다닐정도로또 이병으로 힘들어서 직장을 다니기 힘들정도입니다ㅜㅜ


1 번째 영상의 53 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: 지진웅
3.댓글 작성일자: 11개월 전
4.댓글 내용: 안녕하세요.28살 남자입니다.요식업종사자이며 20kg정도되는 재료들을 10개정도 옮기는과정에서  허리를 삐끗하였고 쭈구려앉아서 해야하는 근무환경이라 초기에 못잡아서 디스크가 왔습니다.7월15일 초기진단을받고 일주일 쉬었다가 다시 근무했는데 8웧7일 재발하였고 걷지도 앉지도못하고 누워만있게됬습니다.일을 쉬라는 판단을받았습니다.퇴행성인지 급성인진 아직 모르는상태이지만 이전에 한번도 이런적이없는데 산재처리 휴업급여 가능할까요??최

3.댓글 작성일자: 1년 전
4.댓글 내용: 질문좀할께요 제 크레인이 넘어가며 함께일하는 기사가 다쳤습니다 그런데 산제로치료해주게되면 별도로 합의를 해야하나요 기사가 저를 경찰서에 고소를 했습니다. 손해배상을 무리하게 요구하기에 저는 그돈을 줄수가없다 했더니 법적으로 한다며 고소했습니다


1 번째 영상의 78 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: fortable com
3.댓글 작성일자: 1년 전
4.댓글 내용: 출근중 교통사고가 나서 산재지정병원에서 입원하고 있다가 산재 신청까지 했는데요. 몇일전에 퇴원했고 일단 진단은 2주 어깨 염좌 등으로 나왔습니다.문제는 어깨가 2주넘게 호전증세없이 계속 아픕니다. 엑스레이,초음파 검사 다했는데 뼈 이상없고 인대가 늘어났지만 수술할정도가 아니라고 했습니다. 근데 너무 계속 아파 병원에 다시 내원해서 mri라도 받아보고 싶은데 이럴경우 산재처리 가능한가요??  mri로 새로운 병명이 추가된다면 산재 소견서쓴건 어떻게 되는건가요????


1 번째 영상의 79 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=iDtglrj7Whg
2.댓글 작성자명: 국의영
3.댓글 작성일자: 1년 전
4.댓글 내용: 질문이 있습니다. 아버지와 회사의 관계는 매우 좋습니다! 아버지가 6월달에 일을 하시다가 눈 쪽을 크게 다치셔서 수술을 2회정도 하시고 아직도 수술이 2회 정도 남아있다고 합니다. 처음에는 회사에서 공상합의로 하자고 하셔서 수술 2회는 공상합의로 진행했습니다. 그런데 이번 달에 아버지에게 산재처리로 할 수도 있다고 합니다. 눈이라서 회사에서 



3 번째 영상의 11 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=AXB2xJSzZ_I
2.댓글 작성자명: 자의누리
3.댓글 작성일자: 5개월 전
4.댓글 내용: 119부르거나 쉬는건 현실적으로 불가능할듯....규모 어느 정도 있는 사업장은 대부분 자체응급실 운영하며 119 신고를 통제하죠그리고 다쳤다고 쉰다면 신체절단급 큰 사고가 아닌 이상 회사에 찍혀서 금방 짤리구요


3 번째 영상의 12 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=AXB2xJSzZ_I
2.댓글 작성자명: 하늘땅
3.댓글 작성일자: 5개월 전
4.댓글 내용: 김노무사님 , 잘. 보겠습니다


3 번째 영상의 13 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=AXB2xJSzZ_I
2.댓글 작성자명: 하늘우주선
3.댓글 작성일자: 1년 전
4.댓글 내용: 고맙습니다


3 번째 영상의 14 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=AXB2xJSzZ_I
2.댓글 작성자명: ppoppi13 7
3.댓글 작성일자: 1년 전
4.댓글 내용: 세번째 관련해서 묻고싶은게 회사에서 나오라고 압박하거나나와서 청소라도 해라 한다면 어떻게 하나요?


3 번째 영상의 15 번째  댓글 
------------------

...

3.댓글 작성일자: 10개월 전(수정됨)
4.댓글 내용: 할아버님 손녀사랑이 주책이십니다ㅋㅋㅋㅋ 보기좋아요~


89 번째 영상의 13 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=Flsk4SbBoH4
2.댓글 작성자명: Katie Kang
3.댓글 작성일자: 10개월 전
4.댓글 내용: 라디오듣고 구독하러 왔어요~  좋은정보 감사합니다 ㅎㅎ


89 번째 영상의 14 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=Flsk4SbBoH4
2.댓글 작성자명: 유성매직.
3.댓글 작성일자: 10개월 전
4.댓글 내용: 아닛 여긴 댓글 청정구역!! 싫어요 도하나도없다니


89 번째 영상의 15 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=Flsk4SbBoH4
2.댓글 작성자명: Sukyoung Oh
3.댓글 작성일자: 10개월 전
4.댓글 내용: 안녕하세요 차산 선생님, 제가 임산부 근로자-스트레스 산재에 관해 궁금한 것이 있어서요. 회사의 사생활침해 관련해서요.우선 배경 내용은... 일전에 팀 内 2년제卒 파견직 후배가 다른 4년제卒 정규직 직원과 밥을 먹고 와서는 식사 중에 들었다는 차별적인 말들에 대해 제게 한탄한 일이 있는데요. 들은 저도 맛있게 먹고 오라고 한 게 후회되고 짠해서 제 개인 인스타에 그 심정을 쓴 일이 있습니다. 그런데 어제 팀장이 저를 불러 개인 면담을 하자더니 그 이야기를 꺼냈습니다. (착상 관련)위험한 시기는 지났죠?, 이런 글





96 번째 동영상의 정보를 수집합니다.
댓글 4개
96 번째 동영상의 조회수는 458 회 이고 수집할 댓글은 총 4개 입니다
96 번째 동영상의 제목은 특수형태 근로종사자의 산재보험 가입방법, 보험료 납부 방법, 적용제외 방법 입니다


96 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


96 번째 영상의 1 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=7E7NRS2rKS0
2.댓글 작성자명: a sk
3.댓글 작성일자: 2일 전
4.댓글 내용: 휴업 또는 휴직처리는 어떤식으로 되나요??


96 번째 영상의 2 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=7E7NRS2rKS0
2.댓글 작성자명: 불타오르네.
3.댓글 작성일자: 2일 전
4.댓글 내용: 대행기사등록할때 산재가입을할것이냐 질문도안하고 그런게있는지도 몰라서 가입을안하고 사고가났습니다. 이럴땐 산재적용을 못받나여?




97 번째 동영상의 정보를 수집합니다.
댓글 0개
97 번째 동영상의 조회

4.댓글 내용: 아직. 3년은. 안. 되었어요?


100 번째 영상의 6 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=e_Lwd4NqQTw
2.댓글 작성자명: 김문성
3.댓글 작성일자: 1주 전
4.댓글 내용: 부족한. 서민들의 소리.  들어주시면. 않될까요?참.  어렵네요,


100 번째 영상의 7 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=e_Lwd4NqQTw
2.댓글 작성자명: 김문성
3.댓글 작성일자: 1주 전
4.댓글 내용: 적응장애 진단받고요.폐절단. 갈비쪽에. 고정핀 7개를.  박았어요


100 번째 영상의 8 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=e_Lwd4NqQTw
2.댓글 작성자명: 김문성
3.댓글 작성일자: 1주 전
4.댓글 내용: 중대제해.  입니다,뉴스에도 크게났던사건입니다,


100 번째 영상의 9 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=e_Lwd4NqQTw
2.댓글 작성자명: 김문성
3.댓글 작성일자: 1주 전
4.댓글 내용: 저희를.   좀. 도와주세요.


100 번째 영상의 10 번째  댓글 
----------------------------------------------------------------------
1.UR

2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~




110 번째 동영상의 정보를 수집합니다.
댓글 2개
110 번째 동영상의 조회수는 6623 회 이고 수집할 댓글은 총 2개 입니다
110 번째 동영상의 제목은 알짜배기 혜택 많은 고용 산재보험 입니다


110 번째 동영상에서 댓글 수집을 시작합니다
댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^


1 번째 페이지의 댓글을 수집하는 중입니다~
2 번째 페이지의 댓글을 수집하는 중입니다~
3 번째 페이지의 댓글을 수집하는 중입니다~
4 번째 페이지의 댓글을 수집하는 중입니다~
5 번째 페이지의 댓글을 수집하는 중입니다~
6 번째 페이지의 댓글을 수집하는 중입니다~
7 번째 페이지의 댓글을 수집하는 중입니다~
8 번째 페이지의 댓글을 수집하는 중입니다~
9 번째 페이지의 댓글을 수집하는 중입니다~
10 번째 페이지의 댓글을 수집하는 중입니다~
11 번째 페이지의 댓글을 수집하는 중입니다~


110 번째 영상의 1 번째  댓글 
----------------------------------------------------------------------
1.URL 주소: https://www.youtube.com//watch?v=O_KrfrkKcNw
2.댓글 작성자명: 도깨비
3.댓글 작성일자: 1년 전
4.댓글 내용: 허리 디스크 문제도 산재가 되나요




111 번째 동영상의 정보를 수집합니다.
댓글 1개
111 번째 동영상의 조회수는 384 회 이고 수집할 댓글은 총 1개 입니다
1



1.요청된 총 200 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 3 건입니다
2.총 소요시간은 27090.4 초 입니다 
3.파일 저장 완료: txt 파일명 : C:\\temp\\산업재해보상보험\\2020-10-16-09-04-41-산업재해보상보험\2020-10-16-09-04-41-산업재해보상보험.txt 
4.파일 저장 완료: csv 파일명 : C:\\temp\\산업재해보상보험\\2020-10-16-09-04-41-산업재해보상보험\2020-10-16-09-04-41-산업재해보상보험.csv 
5.파일 저장 완료: xls 파일명 : C:\\temp\\산업재해보상보험\\2020-10-16-09-04-41-산업재해보상보험\2020-10-16-09-04-41-산업재해보상보험.xls 


In [6]:
              
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )


print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

time.sleep(3)

driver.close( )



1.요청된 총 400 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 64 건입니다
2.총 소요시간은 8706.8 초 입니다 
3.파일 저장 완료: txt 파일명 : C:\\temp\\산업재해보상보험2020-10-15-15-02-54-산업재해보상보험\2020-10-15-15-02-54-산업재해보상보험.txt 
4.파일 저장 완료: csv 파일명 : C:\\temp\\산업재해보상보험2020-10-15-15-02-54-산업재해보상보험\2020-10-15-15-02-54-산업재해보상보험.csv 
5.파일 저장 완료: xls 파일명 : C:\\temp\\산업재해보상보험2020-10-15-15-02-54-산업재해보상보험\2020-10-15-15-02-54-산업재해보상보험.xls 


In [7]:
pwd

'C:\\temp\\산업재해보상보험2020-10-15-15-02-54-산업재해보상보험'